In [28]:
from elasticsearch import Elasticsearch
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement
# Cassandra settings
cluster = Cluster(['127.0.0.1'], port=32769)  # Replace with actual Cassandra cluster address
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS books WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.set_keyspace('books')
session.execute("CREATE TABLE IF NOT EXISTS books.children (trove_id TEXT PRIMARY KEY, title TEXT, url TEXT, contributors LIST<TEXT>, date DATE, format TEXT, pages INT)")
# Elasticsearch settings
es = Elasticsearch(['http://localhost:9200']) 
search_after= None
i=0

session.execute("DROP KEYSPACE IF EXISTS books")
session.execute("CREATE KEYSPACE IF NOT EXISTS books WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};")


insert_query_for_parents = "INSERT INTO books.parents (trove_id,title,url,contributors,date,format,pages) VALUES (?,?,?,?,?,?,?)"
create_table_query_parents= "CREATE TABLE books.parents (\
    title text,\
    url text,\
    contributors text,\
    date text,\
    format text,\
    trove_id text PRIMARY KEY,\
    pages int\
);"


session.execute(create_table_query_parents)
insert_statement = session.prepare(insert_query_for_parents)

res = es.search(
    index='books',
    body={
        "query": {
            "bool": {
                
                "must_not": [
                    {
                        "term": {
                            "parent": ""
                        }
                    }
                ]
            }
        },
        'sort': [{'_doc': 'asc'}],
        'size': 100,
        'search_after': search_after
    }
)

while res['hits']['hits']:
    last_hit = res['hits']['hits'][-1]
    search_after = last_hit['sort']

    for hit in res['hits']['hits']:
        doc_id = hit['_id']
        doc = hit['_source']
        i += 1
        
        row = (
            doc['trove_id'],
            doc['title'],
            doc['url'],
            doc['contributors'],
            doc['date'],
            doc['format'],
            int(doc['pages'])
        )
        session.execute(insert_statement, row)
        print(f"{i} documents have been added to parents table", end="\r")

    res = es.search(
        index='books',
        body={
            "query": {
                "bool": {
                    
                    "must_not": [
                        {
                            "term": {
                                "parent": ""
                            }
                        }
                    ]
                }
            },
            'sort': [{'_doc': 'asc'}],
            'size': 100,
            'search_after': search_after
        }
    )

print(f"\n {i} documents have been uploaded\n")


C:\Users\Pegah\AppData\Local\Temp\ipykernel_14516\4103704326.py:34: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(


C:\Users\Pegah\AppData\Local\Temp\ipykernel_14516\4103704326.py:76: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(


1423 docs added to books.parents.
Total documents: 1423



In [29]:

i=0
#Cassandra Settings:
insert_query_for_children = "INSERT INTO books.children (trove_id,title,url,contributors,date,format,pages) VALUES (?,?,?,?,?,?,?)"
create_table_query_children= "CREATE TABLE books.children (\
    title text,\
    url text,\
    contributors text,\
    date text,\
    format text,\
    trove_id text PRIMARY KEY,\
    pages int\
);"
session.execute(create_table_query_children)
insert_statement = session.prepare(insert_query_for_children)

res = es.search(
    index='books',
    body={
        "query": {
            "bool": {
                
                "must_not": [
                    {
                        "term": {
                            "children": ""
                        }
                    }
                ]
            }
        },
        'sort': [{'_doc': 'asc'}],
        'size': 100,
        'search_after': None
    }
)

while res['hits']['hits']:
    last_hit = res['hits']['hits'][-1]
    search_after = last_hit['sort']

    for hit in res['hits']['hits']:
        doc_id = hit['_id']
        doc = hit['_source']
        i += 1
        # Perform any necessary data transformations or mappings here
        row = (
            doc['trove_id'],
            doc['title'],
            doc['url'],
            doc['contributors'],
            doc['date'],
            doc['format'],
            int(doc['pages'])
        )
        session.execute(insert_statement, row)
        print(f"{i} documents have been added to children table", end="\r")

    res = es.search(
        index='books',
        body={
            "query": {
                "bool": {
                    
                    "must_not": [
                        {
                            "term": {
                                "children": ""
                            }
                        }
                    ]
                }
            },
            'sort': [{'_doc': 'asc'}],
            'size': 100,
            'search_after': search_after
        }
    )

print(f"\n {i} documents have been uploaded\n")


C:\Users\Pegah\AppData\Local\Temp\ipykernel_14516\3402402505.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(


C:\Users\Pegah\AppData\Local\Temp\ipykernel_14516\3402402505.py:60: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(


240 docs added to books.children.
Total documents: 240

